# Thymio Control System

This notebook contains the main control loop for the Thymio robot, integrating vision, state estimation, and motion control modules implemented in separate files.

### Turn off LEDs

In [1]:
import tdmclient.notebook
await tdmclient.notebook.start()  # MAKE SURE TO CALL notebook.stop() ONCE DONE USING THYMIO

In [2]:
%%run_python
nf_leds_bottom_left(0,0,0)
nf_leds_bottom_right(0,0,0)
nf_leds_prox_v(0, 0)
nf_leds_prox_h(0, 0, 0, 0, 0, 0, 0, 0)
nf_leds_circle(0, 0, 0, 0, 0, 0, 0, 0)
nf_leds_temperature(0, 0)


In [3]:
await tdmclient.notebook.stop()

In [4]:
import numpy as np
import vision
from vision import VisionSystem
#from state_estimation import ExtendedKalmanFilter
from control import ThymioController
from pathfinding import find_path
from utils import Pose, Point, euclidean_distance

In [5]:
##### just to get the visu
import vision
from vision import VisionSystem
from pathfinding import find_path
from utils import Point
import importlib
importlib.reload(vision)
import time

try: del vis
except NameError: pass
vis = VisionSystem(camera_index=1, warmup_time=3)
graph, start_node, goal_node = vis.construct_map(cspace_padding=2.0)
waypoints_idx = find_path(graph, start_node, goal_node)
waypoints: list[Point] = [graph.nodes[i]['pos'] for i in waypoints_idx]
vis.live_visu(graph, start_node, goal_node, waypoints, resolution=(1600, 900))



Warming up camera for 3s or until exposure settles...
Camera ready. (Resolution: 1280.0x720.0)
Point(x=np.float32(18.365255), y=np.float32(20.911903))


In [6]:
while True:
    if vis.update_robot_visu():  # returns True if 'q' pressed
        break
    time.sleep(0.05)

### Defining Main Runner Function

In [ ]:
import numpy as np
from tdmclient import ClientAsync
import sys
import time

from vision import VisionSystem 
from control import ThymioController
from pathfinding import find_path
from utils import Pose, Point, euclidean_distance

from extended_kalman_filter import EKF 
from thymio_math_model import Thymio 


INITIAL_POSE = Pose(0, 0, 0)  # x, y, theta
INITIAL_COVARIANCE = np.eye(3) * 1e-1  # Initial uncertainty in state estimation
CAMERA_COVARIANCE = np.diag([
        0.0011857353432198614,
        0.0017873989613467563,
        6.1009773737464586e-05
    ]) # These are from the webcam, they may work but ideally change to Tanc's phone


WAYPOINT_THRESHOLD = 3.0  # cm
DT = 0.05  # Control loop time step in seconds

thymio = Thymio()
thymio.dt = DT
thymio.freq = 1.0 / DT

ekf = EKF(thymio, 
            np.array([INITIAL_POSE.x, INITIAL_POSE.y, INITIAL_POSE.theta]), 
            INITIAL_COVARIANCE,
            CAMERA_COVARIANCE 
           )

async def run(vis: VisionSystem):

    #ekf = ExtendedKalmanFilter(initial_pose=INITIAL_POSE, initial_covariance=INITIAL_COVARIANCE)
    controller = ThymioController()
    
    # B. Map & Plan
    graph, start_node, goal_node = vis.construct_map(cspace_padding=2.0)
    waypoints_idx = find_path(graph, start_node, goal_node)
    waypoints: list[Point] = [graph.nodes[i]['pos'] for i in waypoints_idx]
    
    vis.init_visu(graph, start_node, goal_node, waypoints, resolution=(1400,1000))
    
    current_waypoint_idx = 0
    l_cmd, r_cmd = 0, 0  # Initialize command speeds

    thymio.v_l, thymio.v_r = 0, 0   # Set initial speed commands for thymio_model

    #ekf.predict_step()
    #ekf.update_step()
    
    print("Connectin to Thymio...")

    client = ClientAsync()
    node = await client.wait_for_node()
    await node.lock()
    #await node.stop()

    print("Thymio connected.")
    
    # Loop

    #await node.set_variables({"motor.left.target":[0], "motor.right.target":[0]})
    try:
        print("Starting main loop...")

        while True:
            start_time = time.time()

            await node.wait_for_variables([
                'prox.horizontal',
                'motor.left.target',
                'motor.right.target',
                'motor.left.speed',
                'motor.right.speed'
                ])

            prox_sensors = list(node.v.prox.horizontal)
            # print("Prox Sensors:", prox_sensors)

            # 1. Pose Measurement & Sensing
            vision_pose_measurement = vis.get_robot_pose() # May be None
            print("Vision Pose Measurement:", vision_pose_measurement)
            
            try:
                left_speed_measurement = node.v.motor.left.speed #target
                right_speed_measurement = node.v.motor.right.speed #target


            except Exception as e:
                print(f"Error reading motor speeds: {e}")
                left_speed_measurement = 0
                right_speed_measurement = 0

            
            thymio.v_l = thymio.thymio_unit_to_speed(left_speed_measurement)
            thymio.v_r = thymio.thymio_unit_to_speed(right_speed_measurement)
            
            # 2. State Estimation via EKF
            '''
            if estimated_pose is None:
                print("No pose estimate available, skipping update.")
                continue
            '''
            ekf.predict_step()

            #ekf.predict(robot.left_speed, robot.right_speed, dt)
            #if vision_pose_measurement:
                #ekf.update(vision_pose_measurement)

            if vision_pose_measurement is not None: 
                z = np.array([
                    vision_pose_measurement.x,
                    vision_pose_measurement.y, 
                    vision_pose_measurement.theta
                ], dtype=float)
                
                ekf.update_step(z)
            else: 
                print("No pose estimate available, skipping update.")
                continue

            # EKF estimated pose to Pose()
                
            estimated_pose = ekf.x.flatten() # vision_pose_measurement#ekf.get_state()
            # estimated_pose = Pose(
            #     estimated_pose[0],
            #     estimated_pose[1],
            #     estimated_pose[2]
            #     )
            
            
            # 3. PATH LOGIC
            if current_waypoint_idx < len(waypoints):
                target = waypoints[current_waypoint_idx]
                
                # Check if reached waypoint
                dist_to_wp = euclidean_distance(Point(*estimated_pose[:2]), target)
                if dist_to_wp < WAYPOINT_THRESHOLD:
                    print(f"Reached Waypoint {current_waypoint_idx}!")
                    current_waypoint_idx += 1
                    
                # Check if reached GOAL
                if current_waypoint_idx >= len(waypoints):
                    print("GOAL REACHED! Stopping.")
                    break # Exit loop
                
                # 4. CONTROL
                # Get commands from your controller
                # Ensure controller returns integers! Thymio expects ints.
                l_cmd, r_cmd = controller.update(
                    Pose(estimated_pose[0], estimated_pose[1], estimated_pose[2]),
                    target,
                    prox_sensors
                    )

            # 5. SEND COMMANDS (Async)
            # We write to the variables. This doesn't send yet.
            #node.v.motor.left.target = int(l_cmd)
            #node.v.motor.right.target = int(r_cmd)
            v = {
                    "motor.left.target": [int(l_cmd)],
                    "motor.right.target": [int(r_cmd)],
                }
            print(f"(l_cmd, r_cmd): ({int(l_cmd)}, {int(r_cmd)})")
            await node.set_variables(v)
            
            # Force send the command packet immediately
            node.flush()
            
            # 6. LOOP TIMING 
            # Calculate how much time the logic took, and sleep the remainder
            # to maintain a steady frequency (e.g., 10Hz)
            elapsed = time.time() - start_time
            if elapsed < DT:
                await client.sleep(DT - elapsed)
            
            # Update visu
            if vis.update_robot_visu():  # returns True if 'q' pressed
                print("User requested exit via visu.")
                break

    finally:
        print("Stopping motors and unlocking Thymio...")
        v = {
                    "motor.left.target": [0],
                    "motor.right.target": [0],
                }
        await node.set_variables(v)
        node.flush()
        await node.unlock()

        print("Thymio disconnected.")
        while True:
            if vis.update_robot_visu():  # returns True if 'q' pressed
                break
            await client.sleep(0.05)

    

### Running

##### Vision

In [13]:
try: del vis
except NameError: pass

Camera released.


In [15]:
# Initialize vision system before running (and only once)
vis = VisionSystem(camera_index=1, warmup_time=3)

Warming up camera for 3s or until exposure settles...
Camera ready. (Resolution: 1280.0x720.0)


In [16]:
# Actually run the control system:
try:
    await run(vis)
except KeyboardInterrupt:
    print("Program interrupted by user. Exiting...")

Point(x=np.float32(10.082498), y=np.float32(45.94393))
Connectin to Thymio...
Thymio connected.
Starting main loop...
Vision Pose Measurement: Pose(x=np.float32(10.082498), y=np.float32(45.94393), theta=-1.7004988639508087)
Heading error: -1.07 rad, Linear speed: 57.25, Angular speed: -128.81
(l_cmd, r_cmd): (186, -71)
Vision Pose Measurement: Pose(x=np.float32(10.082498), y=np.float32(45.94393), theta=-1.7004988639508087)
Reached Waypoint 0!
Heading error: -1.40 rad, Linear speed: 20.68, Angular speed: -167.72
(l_cmd, r_cmd): (188, -147)
Vision Pose Measurement: Pose(x=np.float32(9.895206), y=np.float32(45.754993), theta=-1.7640078106427026)
Heading error: 2.28 rad, Linear speed: 0.00, Angular speed: 273.40
(l_cmd, r_cmd): (-273, 273)
Vision Pose Measurement: Pose(x=np.float32(9.895206), y=np.float32(45.723503), theta=-1.8993494973463476)
Heading error: 2.29 rad, Linear speed: 0.00, Angular speed: 275.31
(l_cmd, r_cmd): (-275, 275)
Vision Pose Measurement: Pose(x=np.float32(10.238573)

Exception in thread Exception in thread Thread-5:
Traceback (most recent call last):
  File "c:\Users\johnc\AppData\Local\Programs\Python\Python312\Lib\threading.py", line 1073, in _bootstrap_inner
Thread-4:
Traceback (most recent call last):
  File "c:\Users\johnc\AppData\Local\Programs\Python\Python312\Lib\threading.py", line 1073, in _bootstrap_inner
    self.run()
  File "c:\Users\johnc\AppData\Local\Programs\Python\Python312\Lib\site-packages\tdmclient\tcp.py", line 73, in run
    self.run()
  File "c:\Users\johnc\AppData\Local\Programs\Python\Python312\Lib\site-packages\tdmclient\tcp.py", line 73, in run
    packet = self.read_packet()
             ^^^^^^^^^^^^^^^^^^
  File "c:\Users\johnc\AppData\Local\Programs\Python\Python312\Lib\site-packages\tdmclient\tcp.py", line 66, in read_packet
    packet = self.read_packet()
             ^^^^^^^^^^^^^^^^^^
  File "c:\Users\johnc\AppData\Local\Programs\Python\Python312\Lib\site-packages\tdmclient\tcp.py", line 66, in read_packet
    ra

### Separate Vison Tests

In [18]:
import matplotlib.pyplot as plt

from vision import VisionSystem
from pathfinding import find_path


try: del vis
except NameError: pass
vis = VisionSystem(camera_index=1)

print(vis.get_robot_pose())
plt.imshow(vis.warp_image(vis.img))


# Build graph
g, start, end = vis.construct_map()

# Run the pathfinding
waypoints = find_path(g, start, end)
waypoints = [g.nodes[n]['pos'] for n in waypoints]  # Convert node IDs to world-space

# Visualize the resulting visibility graph
plt.figure(figsize=(16, 10))

# Add the warped image as background to enforce image coordinates (0,0 at top-left)
plt.imshow(vis.warp_image(vis.img)[::-1])

# Since coordinates have been converted to world-space with (0,0) at bottom-left,
# we need to unflip the y-axis (caused by imshow)
plt.gca().invert_yaxis()

# Plot all edges
for u, v in g.edges():
    p1 = g.nodes[u]['pos']
    p2 = g.nodes[v]['pos']
    xs = [p1[0] * vis.pxl_per_cm_x, p2[0] * vis.pxl_per_cm_x]
    ys = [p1[1] * vis.pxl_per_cm_y, p2[1] * vis.pxl_per_cm_y]
    plt.plot(xs, ys, color="lightsalmon", alpha=0.4, linewidth=1)

# Plot the waypoints path
for i in range(len(waypoints) - 1):
    p1 = waypoints[i]
    p2 = waypoints[i + 1]
    xs = [p1[0] * vis.pxl_per_cm_x, p2[0] * vis.pxl_per_cm_x]
    ys = [p1[1] * vis.pxl_per_cm_y, p2[1] * vis.pxl_per_cm_y]
    plt.plot(xs, ys, color="deepskyblue", alpha=0.6, linewidth=5, label="Planned Path" if i == 0 else "")

# Plot all nodes
for i in g.nodes():
    pt = g.nodes[i]['pos']
    color = "deepskyblue" if pt in waypoints else "coral"
    size = 70 if pt in waypoints else 30
    plt.scatter(pt[0] * vis.pxl_per_cm_x, pt[1] * vis.pxl_per_cm_y, color=color, s=size, alpha=0.8, marker="o")

# Highlight start and goal nodes
start_pos = g.nodes[start]['pos']
goal_pos = g.nodes[end]['pos']
plt.scatter(start_pos[0] * vis.pxl_per_cm_x, start_pos[1] * vis.pxl_per_cm_y, color="limegreen", s=120, marker="o", label="Start")
plt.scatter(goal_pos[0] * vis.pxl_per_cm_x, goal_pos[1] * vis.pxl_per_cm_y, color="deeppink", s=120, marker="o", label="Goal")

plt.title("Visibility Graph with Planned Path from Package")
plt.legend()
plt.axis("off")
plt.show()

print(vis.get_robot_pose())
print(vis.get_robot_pose())

Warming up camera for 5s or until exposure settles...


RuntimeError: Camera warmup timed out! (Received only black/empty frames)

In [ ]:
for i in range(20):
    print(f"{vis.get_robot_pose() = }")